In [ ]:
# Standard libraries
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath

import lib.sych.preprocess_raw as prep

%load_ext autoreload
%autoreload 2

# Pooling Data From LVM

In [ ]:
# fpathData = gui_fpath("Root directory for raw data", "./")
# fpathData = '/mnt/neurophys-storage2/Sych/Yaro/data_raw/'
fpathData = '/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/Sych/Yaro/data_raw'

In [ ]:
dfFiles = prep.raw_get_files_df(fpathData)
dfFiles

In [ ]:
prep.raw_pool_data(dfFiles)

# Moving all datasets to data group

In [ ]:
# fpathDataH5 = gui_fpath("Directory for data files", "./")
# fpathDataH5 = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed/'
fpathDataH5 = '/media/aleksejs/DataHDD/work/data/yaro/neuronal-raw-pooled'

In [ ]:
dfRawH5 = prep.pooled_get_files_df(fpathDataH5)

In [ ]:
dfRawH5

In [ ]:
prep.pooled_move_data_subfolder(dfRawH5)

In [ ]:
prep.pooled_move_sanity_check(dfRawH5)

# Mark Starts and ends of Trials

In [ ]:
prep.pooled_mark_trial_starts_ends(dfRawH5)

# Appending Channel Labels

In [ ]:
# fpathDataOrig = gui_fpath("Directory for original data tree", "./")
fpathDataOrig = '/media/aleksejs/DataHDD/work/data/yaro/neuronal/mvg48'

In [ ]:
dfLabels = prep.channel_labels_get_files_df(fpathDataOrig)
dfLabels

In [ ]:
prep.pooled_mark_channel_labels(dfRawH5, dfLabels)

In [ ]:
prep.update_channel_labels_unique(dfRawH5)

# Adding behaviour

**Problems/Bugs**:
1. [early_go_trials, iGO_inhibition] overlap - within this framework could be solved by multiplexing enum
2. mvg_8_2018_11_22_a has 406 trials in behaviour but only 142 in neuro - crop to neuro

**TODO**:
1. Crop trial start/end to match behaviour
2. Drop some trials if Yaro says so

In [ ]:
dfNeuro = prep.orig_neuro_get_files_df(fpathDataOrig)
dfNeuro

In [ ]:
prep.pooled_mark_trial_types_performance(dfRawH5, dfNeuro)

# Drop bad sessions

1. Drop sessions that were not used by Yaro in first analysis

In [ ]:
prep.drop_sessions_not_in_neuro(dfNeuro, dfRawH5)

2. Deleting extra sessions that behave badly

In [ ]:
prep.drop_session(dfRawH5, 'mvg_8_2018_11_22_a')
prep.drop_session(dfRawH5, 'mvg_9_2019_03_21_a')

1. Filtering short trials

In [ ]:
shortTrialDict = prep.find_short_trials(dfRawH5)

In [ ]:
shortTrialDict

In [ ]:
for session, idxsTrial in shortTrialDict.items():
    prep.drop_trials(dfRawH5, session, idxsTrial)

3. Filtering outlier trials - orders of magnitude larger than others

In [ ]:
largeTrialDict = prep.find_large_trials(dfRawH5)

In [ ]:
largeTrialDict

In [ ]:
for session, idxsTrial in largeTrialDict.items():
    prep.drop_trials(dfRawH5, session, idxsTrial)

# Cropping starts and ends of trials to match behaviour

In [ ]:
prep.pooled_trunc_trial_starts_ntrials(dfRawH5)

# Testing trial length sensibility

In [ ]:
prep.pooled_trial_length_summary_excel(dfRawH5)

# Pulling Trial Durations From Behaviour Files


In [ ]:
dfBehavTiming = prep.behav_timing_get_files_df(fpathData)
dfBehavTiming

In [ ]:
prep.behav_timing_read_get_trial_lengths(dfRawH5, dfBehavTiming)

In [ ]:
prep.behav_timing_compare_neuro(dfRawH5)

# Background Subtraction

Test if there is significant gain from using higher order polynomials

In [ ]:
prep.pooled_plot_background_polyfit_residuals(dfRawH5, ordMax=8)

In [ ]:
%matplotlib notebook
# %matplotlib inline
prep.poly_view_fit(dfRawH5, 'mvg_7_2018_11_26_a', 21, 2, onlyTrials=False, onlySelected=True)

# Test if pre-trial activity is actually small

In [ ]:
prep.check_pre_trial_activity_small(dfRawH5)

# Baseline Normalization

* ZScore (Trial/Session-wise)
* Pre-trial mean estimate (Trial/Session-wise)

In [ ]:
prep.extract_store_trial_data(dfRawH5, targetFPS=20, bgOrd=15, fitOnlySelectedTrials=True, keepExisting=False)

In [ ]:
for idx, row in dfRawH5.iterrows():
    with h5py.File(row['path']) as h5file:
        for session in h5file['data_raw'].keys():
            print(session,
                  len(h5file['trialTypes'][session]),
                  len(h5file['trialStartIdxs'][session]),
                  len(h5file['trialTypesSelected'][session]),
                  h5file['data_raw'][session].shape)

In [ ]:
for idx, row in dfRawH5.iterrows():
    with h5py.File(row['path']) as h5file:
        for session in h5file['data'].keys():
            data = h5file['data'][session]
            
            plt.figure()
            plt.plot(data[:, 3])
            plt.show()
            
            break
    break

# Extract and Store Performance